# Python AlgoTrader

### Created By: Aaron Gao

Python AlgoTrader is an automated trading system that executes orders based on the crossover of the 50 minute and 200 minute moving average. By searching for the iconic Golden Cross and the infamous Death Cross, intraday traders can find great opportunities by riding out the momentum on breakout trends.

We will first load all of the data required for this project through the Python script load_data.py. We will be using the Yahoo Finance API yfinance to gather a financial database containing the 1 minute interval weekly data for each consituent of the S&P 500. The tickers are located in the file 'tickers.csv'.

In [ ]:
# Import Python Libraries
import yfinance as yf
import csv
import os
import shutil

# Start Date
start_date = '2024-09-23'
# End Date
end_date = '2024-09-27'
# Time Interval
interval = '1m'

# Remove existing stock data if it exists
if os.path.exists('historical_data'):
    shutil.rmtree('historical_data')

# Creates historical_data folder to store stock data
os.makedirs('historical_data')

# Read tickers of S&P 500 holdings from tickers.csv file and store the stock data
# retrieved from Yahoo Finance into .csv files in historical data folder
with open("tickers.csv") as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        ticker = row[0]
        
        # Format the file name to contain all the relevant information on a stock's historical data including
        # the ticker, the start date of data, the end date of data, and the interval of which the data is being
        # retrieved. This way the relevant data can easily be accessed in the generate_trades.py file
        file_name = ticker + '_' + start_date + '_' + end_date + '_' + interval + '.csv'
        
        stock_data = yf.download(ticker, start_date, end_date, interval=interval, prepost=False)
        stock_data.to_csv('historical_data/' + file_name)

Once the data is loaded into the historical data folder as comma seperated values with each file following the form '[TICKER]_[START_DATE]_[END_DATE].csv', we are ready to load the trading algorithm generate_trades.py. 

Some of the libraries we will be using include:

In [2]:
# Import Python Libraries
import csv
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib.backends.backend_pdf import PdfPages
from datetime import datetime
import os
import shutil

Where csv will be used to read our stored historical data, plt will be used to plot price by volume charts, pd will be used to create DataFrames for our tables, PdfPages will be used to place our chart and tables into a downloadable pdf, datetime will be used to format the date for our time series, and os will be used to access files/folders within our project.

In [3]:
# Check if trade_reports folder exists and remove old financial reports 
if os.path.exists('trade_reports'):
    shutil.rmtree('trade_reports')

# Create new trade_reports folder to store new financial reports
os.makedirs('trade_reports')    

# Loop through all csv files in historical data folder
directory = os.fsencode('historical_data')

This is the beginning of our program where we will loop through all the .csv files storing the historical data of each individual stock.

In [ ]:
for file in os.listdir(directory):

These are the variables we will be using to generate the trades for each security.

In [ ]:
file_name = os.fsdecode(file)

# Ticker, Start Date, End Date, Interval retrieved from .csv file name
ticker = file_name.split('_')[0]
start_date = datetime.strptime(file_name.split('_')[1], '%Y-%m-%d').date()
end_date = datetime.strptime(file_name.split('_')[2], '%Y-%m-%d').date()
interval = file_name.split('_')[3].split('.')[0]

# Arrays to store data
data = []
dates = []
price = []
volume = []

# 50 minute moving average
ma_50_minute_price_buildup = 0
ma_50_minute_dates = []
ma_50_minute_price = []

# 200 minute moving average
ma_200_minute_price_buildup = 0
ma_200_minute_dates = []
ma_200_minute_price = []

# Intersection Points
golden_cross_dates = []
golden_cross_price = []
death_cross_dates = []
death_cross_price = []

# Important Characteristics: Min Price, Max Price, Min Volume, Max Volume
minPrice = 1e9
minPriceDate = -1
maxPrice = 0
maxPriceDate = -1
minVolume = 1e15
minVolumeDate = -1
maxVolume = 0
maxVolumeDate = -1

# Market value of position throughout trading 
initial_market_value = 1000000
market_value = initial_market_value
long_position = 0
short_position = 0

# Transaction History to store trades
transaction_history = []

Read through the historical data within the csv file for each security and transfer the information to its respective array.

In [ ]:

# Read historical data stored within csv file and transfer into python arrays
with open('historical_data/'+file_name) as csv_file:
    csv_reader = csv.reader(csv_file, delimiter=',')
    for row in csv_reader:
        # Skip first line in .csv file which is a header that describes each column
        if (row[1] == 'Open'): continue

        # Add data to the respective arrays
        dates.append(row[0])
        price.append(float(row[1]))
        volume.append(float(row[6]))


Calculate the maximum and minimum price and volume of the historical data.

In [ ]:
for i in range(len(dates)):
    # Calculate maximum and minimum price
    if minPrice > price[i]:
        minPrice = price[i]
        minPriceDate = dates[i]
    if maxPrice < price[i]:
        maxPrice = price[i]
        maxPriceDate = dates[i]
        
    # Calculate maximum and minimum volume
    if minVolume > volume[i]:
        minVolume = volume[i]
        minVolumeDate = dates[i]
    if maxVolume < volume[i]:
        maxVolume = volume[i]
        maxVolumeDate = dates[i]

Calculate the 50 minute and 200 minute moving averages.

In [ ]:
# Calculate 50 minute moving average
ma_50_minute_price_buildup += price[i]/50
# Check that over 50 minutes have elapsed
if i >= 50:
    ma_50_minute_price_buildup -= price[i-50]/50
    ma_50_minute_price.append(ma_50_minute_price_buildup) 
    ma_50_minute_dates.append(dates[i])
    
# Calculate 200 minute moving average
ma_200_minute_price_buildup += price[i]/200
# Check that over 200 minutes have elapsed
if i >= 200:
    ma_200_minute_price_buildup -= price[i-200]/200
    ma_200_minute_price.append(ma_200_minute_price_buildup)
    ma_200_minute_dates.append(dates[i])

Look for the Golden Cross (Buy/Cover Signal) and Death Cross (Sell/Short Signal) among the data.

In [ ]:
# Golden Cross: 50 minute MA moves above 200 minute MA
if ma_50_minute_price[i-50-1] < ma_200_minute_price[i-200-1] and ma_50_minute_price_buildup > ma_200_minute_price_buildup:
    # Record transaction date and price for golden cross event
    golden_cross_dates.append(dates[i])
    golden_cross_price.append(ma_50_minute_price_buildup)
    
    # Buy Signal if there are no short positions
    if short_position == 0:
        long_position = market_value/price[i]
        transaction_history.append([dates[i], market_value, "Buy", price[i], long_position])
        market_value = 0
    # Cover Signal if there are short positions
    else:
        market_value -= short_position*price[i]-market_value
        short_position = 0
        transaction_history.append([dates[i], market_value, "Cover", price[i], market_value/price[i]])
            
# Death Cross: 50 minute MA moves below 200 minute MA
if ma_50_minute_price[i-50-1] > ma_200_minute_price[i-200-1] and ma_50_minute_price_buildup < ma_200_minute_price_buildup:
    # Record transaction date and price for death cross event
    death_cross_dates.append(dates[i])
    death_cross_price.append(ma_50_minute_price_buildup)
    
    # Sell Signal if there is a long positions
    if long_position == 0:
        short_position = market_value/price[i]
        transaction_history.append([dates[i], market_value, "Short", price[i], short_position])
    # Short Signal if there are no long positions
    else:
        market_value = long_position*price[i]
        long_position = 0
        transaction_history.append([dates[i], market_value, "Sell", price[i], market_value/price[i]])

Setup the chart and table to be placed into a downloadable pdf document.

In [ ]:
pdf = PdfPages('trade_reports/'+file_name.split('.')[0]+'.pdf')
fig = plt.figure(dpi=100)

# Plot Charts. Grey = Price. Orange = 50 minute MA. Blue = 200 minute MA. 
plt.title(ticker + " " + str(start_date) + " " + str(end_date) + " " + interval)

# ax1 subplot will be used to plot all price related graphs
ax1 = plt.subplot()
ax1.set_xlabel('Date', fontsize=8)
ax1.set_ylabel('Price', fontsize=8)

# Give graph some room at the bottom and top to prevent entire chart from being spread too wide
ax1.set_ylim(0.995*minPrice, 1.005*maxPrice)
ax1.yaxis.set_tick_params(labelsize=6)

# ax2 subplot will be used to plot all volume related graphs
ax2 = ax1.twinx()
ax2.set_ylabel('Volume', fontsize=8)
ax2.set_ylim(0, 5*maxVolume)
ax2.yaxis.set_tick_params(labelsize=6)

# Plot stock price
ax1.plot(dates, price, color='silver', linewidth=0.75) 
# Plot 50 minute moving average line
ax1.plot(ma_50_minute_dates, ma_50_minute_price, color='darkorange', linewidth=0.75)
# Plot 200 minute moving average line
ax1.plot(ma_200_minute_dates, ma_200_minute_price, color='darkblue', linewidth=0.75)
# Plot golden cross intersection points
ax1.plot(golden_cross_dates, golden_cross_price, 'g^', markersize=3)
# Plot death cross intersection points
ax1.plot(death_cross_dates, death_cross_price, 'rv', markersize=3)

ax2.bar(dates, volume, width=1.0, color='cornflowerblue')

ax1.xaxis.set_visible(False)

pdf.savefig(fig)

# Check if there are intersections of the moving averages. If there are no intersections, do not
# add a table in the second page detailing the intersection dates. 
if (len(transaction_history) > 0):
    plt.clf()
    plt.title(ticker + " " + str(start_date) + " " + str(end_date) + " " + interval)

    df = pd.DataFrame(transaction_history)    
    df.columns = ["Date", 'Market Value', 'Action', 'Price', '# Shares']

    plt.axis('off')
    plt.table(cellText=df.values, colLabels=df.columns, loc='center')

    pdf.savefig(fig)
    
pdf.close()